In [8]:
import boto3
import csv, json

In [400]:
s3 = boto3.client("s3")
comprehend = boto3.client('comprehend')
dynamodb = boto3.resource('dynamodb')
athena_client = boto3.client('athena')

In [60]:
bucketname = 'public-datasets-multimodality'
filename='json-data/year=2021/month=05/day=12/'
fileObjds = s3.list_objects(Bucket=bucketname, Prefix='json-data/year=2021/')
fileObj = s3.get_object(Bucket=bucketname, Key='json-data/year=2021/month=05/day=12/hour=18twitter-1-2021-05-12-18-10-00-7bcd7d5d-56a1-4c55-b982-439be31e42b1')

In [61]:
fileObj

{'ResponseMetadata': {'RequestId': 'VXCNBVE9S0KTDBK5',
  'HostId': 'ZXPKo8nNin4rDFw6qkvtneAH+I/+ykVUDH4n8JkFz+U+DoQspOLqZJWUmGUMPNdsVgvO+6SDE58=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZXPKo8nNin4rDFw6qkvtneAH+I/+ykVUDH4n8JkFz+U+DoQspOLqZJWUmGUMPNdsVgvO+6SDE58=',
   'x-amz-request-id': 'VXCNBVE9S0KTDBK5',
   'date': 'Wed, 12 May 2021 18:15:38 GMT',
   'last-modified': 'Wed, 12 May 2021 18:15:02 GMT',
   'etag': '"12157751367a340db8d160b5695aadd0"',
   'accept-ranges': 'bytes',
   'content-type': 'application/octet-stream',
   'content-length': '439939',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2021, 5, 12, 18, 15, 2, tzinfo=tzutc()),
 'ContentLength': 439939,
 'ETag': '"12157751367a340db8d160b5695aadd0"',
 'ContentType': 'application/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f9a1b0ff4f0>}

In [62]:
file_content = fileObj["Body"].read().decode('utf-8').splitlines(True)

In [56]:
file_content

['1392529125974241281,2021-05-12 10:53:06 PDT,-700,nan,@25RIVER0AD Unfortunately it’s still not giving me access,en1392487993210900482,2021-05-12 10:53:04 PDT,-700,nan,@Daffantin Fendi x erigo,es1392509874563698693,2021-05-12 10:52:54 PDT,-700,nan,@bodega 🤞This would be a great first raffle win 🙏,en1392538271461396481,2021-05-12 10:52:50 PDT,-700,nan,@tnatw after buying slido cisco makes another purchase in the event space  https://t.co/hG4e11EnwQ,en1392529125974241281,2021-05-12 10:52:46 PDT,-700,nan,@cisco_18_23 Did you get them?,en1392319057014272007,2021-05-12 10:52:29 PDT,-700,nan,@davilagre Esooo  https://t.co/M40G4CWXfw,und1392538146869596162,2021-05-12 10:52:20 PDT,-700,nan,Cybersecurity grows more challenging for government agencies every day and the sophistication of threats continues to rise. Check out this Q&amp;A session by Cisco &amp; Govtech to learn more! @CiscoGateway @CiscoSecure  https://t.co/wPmL8wyzls,en1392538108252545024,2021-05-12 10:52:11 PDT,-700,nan,Ai sincer

In [134]:
reader = csv.reader(file_content, delimiter='|')
table = dynamodb.Table('tweetSentiment')
language_list=['ar', 'hi', 'ko', 'zh-TW', 'ja', 'zh', 'de', 'pt', 'en', 'it', 'fr', 'es']
for row in reader:
    print(len(row))
    if len(row)<4:
        continue
    language=row[5] if len(row)==6 else 'en'
    language='en' if language not in language_list else 'en'
    print(language)
    table.put_item(
            Item={
                'Created_at': row[1],
                'ID': row[0],
                'tweet': row[4],
                'Sentiment': comprehend.detect_sentiment(Text=row[4],LanguageCode=language)['Sentiment']
                }
            )   

6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en
6
en


KeyboardInterrupt: 

In [89]:
credentials = boto3.Session().get_credentials()

In [90]:
credentials

In [178]:
comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
translate = boto3.client(service_name='translate', region_name='us-west-2')

In [182]:
output=translate.translate_text(Text='北京长城你好', SourceLanguageCode='zh', TargetLanguageCode="en")['TranslatedText']

In [185]:
json.dumps(comprehend.detect_entities(Text=output, LanguageCode='en'))

'{"Entities": [{"Score": 0.4298441410064697, "Type": "LOCATION", "Text": "Beijing", "BeginOffset": 6, "EndOffset": 13}, {"Score": 0.7490626573562622, "Type": "LOCATION", "Text": "Great Wall", "BeginOffset": 14, "EndOffset": 24}], "ResponseMetadata": {"RequestId": "c0052e3c-a2b5-488d-af68-6997aa05e04b", "HTTPStatusCode": 200, "HTTPHeaders": {"x-amzn-requestid": "c0052e3c-a2b5-488d-af68-6997aa05e04b", "content-type": "application/x-amz-json-1.1", "content-length": "208", "date": "Fri, 14 May 2021 16:48:07 GMT"}, "RetryAttempts": 0}}'

In [133]:
import pandas as pd
text='To determine the named entities in a document, use the Amazon Comprehend DetectEntities operation. To detect Cisco entities in upto 25 reInvent documents in a batch, use the BatchDetectEntities operation'
ner=json.dumps(comprehend.detect_entities(Text=text, LanguageCode='en'))
nerr=json.loads(ner)

In [151]:
entities=''
for nn in nerr['Entities']:
    entities += nn['Text']+','

In [152]:
entities

'Amazon,Comprehend,Cisco,upto 25 reInvent documents,'

In [208]:
from requests_aws4auth import AWS4Auth
credentials = boto3.Session().get_credentials()

In [209]:
credentials.token

In [210]:
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region='us-west-2', service='es')

TypeError: 

In [212]:
#host='https://vpc-tweetsentiment-55lbq3n5psi2zzp7dw7qhuwqpu.us-west-2.es.amazonaws.com'
host='https://search-twitter-c4idtc5xplyaujgttxrwhyoede.us-west-2.es.amazonaws.com'
url = host + '/sentiment' + '/' + '_doc' + '/'

In [213]:
document={"Created_at": {"S": "2021-05-12 10:20:58 PDT"}, \
 "ID": {"S": "1392530252090560514"}, \
 "Sentiment": {"S": "NEGATIVE"}, \
 "tweet": {"S": "now you apologize, that\'s what they all say!"}}
document

{'Created_at': {'S': '2021-05-12 10:20:58 PDT'},
 'ID': {'S': '1392530252090560514'},
 'Sentiment': {'S': 'NEGATIVE'},
 'tweet': {'S': "now you apologize, that's what they all say!"}}

In [216]:
print(document['ID']['S'])
headers = { "Content-Type": "application/json" }

1392530252090560514


In [218]:
import requests
requests.put(url + document['ID']['S'], auth=awsauth, json=document, headers=headers)

<Response [403]>

In [219]:
from requests_aws4auth import AWS4Auth
region='us-west-2'
service='es'
url='https://search-twitter-c4idtc5xplyaujgttxrwhyoede.us-west-2.es.amazonaws.com/twitter'

In [200]:
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

In [201]:
requests.get(url, auth=awsauth)

<Response [403]>

In [206]:
ata="saddddddddddddddddddddddddddddddddddddd"
info = (ata[:7]) if len(ata) > 7 else ata

In [207]:
info

'saddddd'

In [221]:
from elasticsearch import Elasticsearch

In [225]:
es=Elasticsearch(['https://search-twitter-c4idtc5xplyaujgttxrwhyoede.us-west-2.es.amazonaws.com/'], http_auth=('mltest', 'take2tEZ!'))

In [285]:
es.indices.create(index='twittersentiment')

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'twittersentiment'}

In [236]:
indices_dict = es.indices.get_alias("*")
print ("\nindices_dict type:", type(indices_dict))
for index_name, value in indices_dict.items():
    print ('index: ', value)


indices_dict type: <class 'dict'>
index:  {'aliases': {'.kibana': {}}}
index:  {'aliases': {'.kibana_-1071093071_mltest': {}}}
index:  {'aliases': {}}
index:  {'aliases': {}}


In [289]:
es.indices.delete(index='tweetsentiment')

{'acknowledged': True}

In [247]:
#to get index names only. It will be Python keys with no additional information.
es.indices.get_alias().keys()

dict_keys(['.kibana_1', '.kibana_-1071093071_mltest_1', '.opendistro_security', 'sentiment'])

In [283]:
res = es.index(index="tweetsentiment", id=1, body=_doc)
print(res['result'])

created


In [242]:
res = es.get(index="sentiment", id=1)
print(res['_source'])

NotFoundError: NotFoundError(404, '{"_index":"sentiment","_type":"_doc","_id":"1","found":false}')

In [290]:
_doc = {
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "ID": {
                "type": "text" # formerly "string"
            },
            "Created_at": {
                "type": "date"
            },
            "Entities": {
                "type": "text"
            },
            "Location": {
                "type": "text"
            },
            "Sentiment": {
                "type": "text"
            },
            "Tweet": {
                "type": "text"
            }
        }
    }
}

In [291]:
# make an API call to the Elasticsearch cluster
# and have it return a response:
response = es.indices.create(
    index="sentiment",
    body=_doc,
    ignore=400 # ignore 400 already exists code
)

In [292]:
if 'acknowledged' in response:
    if response['acknowledged'] == True:
        print ("INDEX MAPPING SUCCESS FOR INDEX:", response['index'])

# catch API error response
elif 'error' in response:
    print ("ERROR:", response['error']['root_cause'])
    print ("TYPE:", response['error']['type'])

# print out the response:
print ('\nresponse:', response)

INDEX MAPPING SUCCESS FOR INDEX: sentiment

response: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'sentiment'}


In [3]:
import datetime
datetime.datetime.now()

datetime.datetime(2021, 5, 17, 11, 46, 26, 958602)

In [40]:
doc_body = {
"ID": "1392267665884938243",
"Created_at": datetime.datetime.now(),
"Entities": "KayceeMaduYEG",
"Location": "nan",
"Sentiment": "NEGATIVE",
"Tweet": "@KayceeMaduYEG Ass!"
}
type(doc_body)

dict

In [299]:
DOC_ID =1
res = es.index(index="sentiment", doc_type = '_doc', id = DOC_ID, body = doc_body, request_timeout=45)
print(res['result'])

created


In [303]:
str1="2021-05-12 17:02:56 PDT".split(' ')
datetime.datetime.fromisoformat(str1[0]+'T'+str1[1])

datetime.datetime(2021, 5, 12, 17, 2, 56)

In [322]:
# delete a document from an index
es.delete(index="sentiment",doc_type='_doc',id='1393328441731260416')

{'_index': 'sentiment',
 '_type': '_doc',
 '_id': '1393328441731260416',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 902,
 '_primary_term': 1}

In [343]:
# Josn sub_dict
f = open('outputs/tweets_cisco.1.json',)
jdata = json.load(f)
keys_to_extract = ["id", "created_at", "place", "language", "tweet"]
sub_jdata={key: jdata[key] for key in keys_to_extract}

In [344]:
sub_jdata

{'id': 1389291720932724736,
 'created_at': '2021-05-03 11:52:12 PDT',
 'place': '',
 'language': 'en',
 'tweet': 'Low-power Wide Area Networks Market Challenges during COVID-19 Pandemic, Economic Growth by Players – AT＆T Inc., Qualcomm Inc., Huawei Technologies Co. Ltd., WAVIoT, Cisco Systems, Telefonic... #Sigfox #covid via  https://t.co/sVkGN6swOQ  https://t.co/wHUUC6RYPr'}

In [402]:
#CSV sub df
import numpy as np
def isNaN(str):
    str != str
def to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]
file_content = open('2.csv')
col_list = ["conversation_id","created_at","timezone","place","tweet","language"]
data = pd.read_csv(file_content, usecols=col_list)


to_str(data['conversation_id'][0]) 

'1393711577392730117'

In [401]:
dynamodb_tablename = 'media_sensor_etl_queries'
#SQL for ETL from staging into Parquet table
sql_insert_into = ('insert into demodb.tweetsentiment' 
    '(deviceid, datetime, temperature, humidity, winddirection, windintensity, rainheight, day) '
    'select deviceid, datetime, temperature, humidity, winddirection, '
    'windintensity, rainheight, date_format(datetime, \'%Y-%m-%d\') as day '
    'from iot_data_staging ')
def load_from_staging():
    
    response = athena_client.start_query_execution(
        QueryString=sql_insert_into,
        QueryExecutionContext={
            'Database': glue_database
        },
        ResultConfiguration={
            'OutputLocation': query_output_location
        },
        WorkGroup='primary'
    )

    query_id = response['QueryExecutionId']
    
    #Write the query id into DynamoDB so we can check it for success next time
    update_dynamodb(query_id)

#Update the single item in the DynamoDb table
def update_dynamodb(query_id_or_status):
    
    dynamodb.put_item(
        TableName = dynamodb_tablename, 
        Item =  {'query_execution_id': {
                    'S': 'query_id',
                },
                 'value' : {
                    'S': query_id_or_status,
                }
        }
    )    

In [33]:
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['conversation_id']
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [53]:
with open(r't3.json', 'w', newline='') as file:
    with open(r't3.csv', 'r') as data:
        for line in csv.DictReader(data):
            file.write(str(line)+' ')

In [49]:
with open(r't3.json', 'w', newline='') as file:
    file.write(str(lines))

In [34]:
csvFilePath = r't3.csv'
jsonFilePath = r't3.json'
csvFilePath
# Call the make_json function
make_json(csvFilePath, jsonFilePath)

In [17]:
import uuid
str(uuid.uuid4())
csvFilePath = csvFilePath.csv'

In [31]:
ll=[["SN", "Name", "Contribution"]]
ll.append([1, "Linus Torvalds", "Linux Kernel"])
ll.append([2, "John Travota", "Staturdaynight Live"])
print(ll)
type(ll)

[['SN', 'Name', 'Contribution'], [1, 'Linus Torvalds', 'Linux Kernel'], [2, 'John Travota', 'Staturdaynight Live']]


list

In [39]:
dd={
        "id": "1394390351532761100",
        "created_at": "2021-05-17 13:32:20",
        "location": "-",
        "sentiment": "POSITIVE",
        "entities": "Cisco #FutureVoices @CiscoGateway https://t.co/dB3Bxkxz5n ",
        "tweet": "Welcome to Cisco #FutureVoices \u2013 a platform that champions the voices of the educators and the students, giving airtime to some of the most interesting future-shaping projects in the world of education. @CiscoGateway  https://t.co/dB3Bxkxz5n"
    },
   {
        "id": "1394319544919937026",
        "created_at": "2021-05-17 13:31:54",
        "location": "-",
        "sentiment": "NEGATIVE",
        "entities": "CTVCalgary ",
        "tweet": "@CTVCalgary Much too late. I guest that is why we get the myth-busting tour of your data sets and a plea for participatory government? Once the heat is off, you\u2019ll return to demanding Adversarial government and continue to abuse majority rule.  It\u2019s all about staying in power, isn\u2019t it?"
    }

type(dd)

IndentationError: unexpected indent (<ipython-input-39-682571bdb36f>, line 9)